# **10.10 Python tutorial 3 - <br> Solving material balances with linear algebra in python**

This tutorial builds upon your Python skills from Problem Set 1 (basic Python, NumPy arrays, Matplotlib plotting) and Problem Set 2 (functions, file I/O, loops, conditionals). Here, you will learn how to set up and solve systems of linear equations using `numpy`, a powerful Python library for numerical operations. This is a fundamental skill for solving many chemical engineering problems, like complex material and energy balances.  
<br>

**Learning objectives for this tutorial**  

By the end of this tutorial, you will be able to:  


*   Formulate material balance equations as a system of linear equations.
*   Implement a Python function (such as `equations_function(X)`) that represents these equations.
*   Use your function to verify a known solution (e.g., result from a hand calculation or your friends' calculations).
*   Programmatically construct the coefficient matrix (A) and the constants vector (b) from your `equations_function`.
*   Solve the system of linear equations using `np.linalg.solve` from the `numpy` library
*   Interpret the results and understand the role of numerical precision, conceptually.

<br>

**Prerequisite knowledge**  


*   Basic Python syntax (variables, operators, print statements)
*   NumPy basics (creating arrays)
*   Defining and calling Python functions (from Tutorial 2)
*   Conditional statements (if/elif/else) and for loops (from Tutorial 2)
*   Fundamental understanding of material balances (steady-state, with reaction)



## Step 1: Material Balances as Linear Equations

Many engineering problems, particularly material and energy balances at steady-state, can be expressed as a system of linear equations. For example, consider two unknown molar flow rates in stream 3, $n_{3,A}$ and $n_{3,B}$, and two constraints on them (of unknown origin, this is just to illustrate math and Python at this point).

Equation 1:  $\quad a_{11} \cdot n_{3,A} + a_{12} \cdot n_{3,B} = b_1$  
Equation 2: $\quad a_{21} \cdot n_{3,A} + a_{22} \cdot n_{3,B} = b_2$  

Where $a_{ij}$ are coefficients and $b_i$ are constants.  
(for example a combustion problem where $b_i$ are the flowrates of the reaction products and $a_{ij}$ are stoichiometric ratio -- ah ok this makes sense as a cheme problem now!) <br>


In matrix form, this becomes $A \cdot X = B$, where
$A = \begin{bmatrix}  a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix},\quad  X = \begin{bmatrix} n_{3,A} \\ n_{3,B}  \end{bmatrix}, \quad  B = \begin{bmatrix}b_1 \\ b_2  \end{bmatrix}$  
<br><br>
Solving this system involves finding the values in the $X$ vector that satisfy both equations.  
<br><br>

## Step 2: The equations_function(X) Approach that 10.10 Uses

Define a Python function that takes a NumPy array x (representing an array of unknown solution values) and returns a NumPy array `eq`. Each element `eq[i]` in the array `eq` represents the value of the i-th **equation** when the terms are rearranged such that the equation should be zero when the correct solution x is provided. This approach clarifies the definition of the equations from the solving mechanism.  
<br>
Example by hand: Solve for x and y:  
Equation 1: $2x + 3y = 7 \quad$ Equation 2: $x - y = 1$  

First, rearrange to the form where the right side is zero (this often is differential equation form for our balances):  
Equation 1: $2x + 3y - 7 = 0 \quad$ Equation 2: $x - y - 1 = 0$  

By hand if you subtract $2\times Equation 2$, from $Equation 1$:<br>
$Equation 1 - 2\times Equation 2 = (2x + 3y - 7) - 2\times(x - y - 1) = 5y - 5 = 0$  
which gives $y = 1, x = 2$ (by substituting y = 1 into either equation)
<br>
Now, define the Python function with equations in the rearranged (differential equation for balances) forms:

In [1]:
import numpy as np

def simple_equations(X_values):
    """
    Represents the system of linear equations:
    2x + 3y = 7
    x - y = 1

    Args:
        X_values (np.ndarray): A 1D NumPy array representing the unknowns [x, y].

    Returns:
        np.ndarray: A 1D NumPy array containing the residuals of each equation.

    Operations:
        (1) first, distributes X_values to the variables we use in our equations
        (2) calculate the "residual" (i.e. rearranged or differential form) equations
        (3) return a single vector with all the residuals

    Key Note:
        Check to make sure # of equations = # of unknowns!
    """
    x = X_values[0] # First element of X_values is 'x'
    y = X_values[1] # Second element of X_values is 'y'

    eq1_val = 2*x + 3*y - 7 # Equation 1 arranged to equal zero
    eq2_val = x - y - 1     # Equation 2 arranged to equal zero

    return np.array([eq1_val, eq2_val])

# Test the function with some arbitrary values
test_X = np.array([2.0, 1.0]) # This is the hand solution for x and y
residuals = simple_equations(test_X)
print(f"Residuals for X = {test_X}: {residuals}")

# Try values that are not the solution
test_X_wrong = np.array([0.0, 0.0])
residuals_wrong = simple_equations(test_X_wrong)
print(f"Residuals for X = {test_X_wrong}: {residuals_wrong}")

Residuals for X = [2. 1.]: [0. 0.]
Residuals for X = [0. 0.]: [-7. -1.]


When `X_values` is the correct solution `[2, 1]`, the function returns `[0. 0.]`, indicating the equations are satisfied and the tested solution is correct. When it's not the solution, non-zero values are returned.  Turns out this function is very helpful to us.
<br><br>

## Step 3: Verification of the Manual Solution

After manually solving the problem, verify the answer using this function.  


*   Create a NumPy array with the manual solution.
*   Call equations_function with this array.
*   The returned values should be extremely close to zero. They may not be exactly zero due to floating-point precision issues. Values like 1e-15 or -2.3e-16 are essentially zero.


In [2]:
# Assuming you solved manually and found x=2, y=1
X_manual = np.array([2.0, 1.0])
residuals_manual = simple_equations(X_manual)
print(f"\nVerification of Manual Solution (all results of rearranged equations should be near zero): {residuals_manual}")


Verification of Manual Solution (all results of rearranged equations should be near zero): [0. 0.]


## Step 4: Programmatic Construction of Matrix A and Vector b

This is a powerful numerical technique!  
Instead of manually typing out the A and b matrices manually, use the `equations_function` to derive them.  <br>

A system of linear equations can be written as
$A \cdot X - B = 0$, where  
`equations_function(X)` returns the vector array resulting from the operation $A \cdot X - B $, and give you a vector of $0$'s when $X$ is the solution.<br><br>
Here's a key concept: you get a result of $0$ when $X$ is the solution, but you still get values `equations_function(X)` no matter what $X$ you call it with.
<br>
As discussed in the **required reading**, you recall that we can carefully feed test values for $X$ consisting of 0's and 1's to determine $A$ and $B$.  Or more precisely, B and A.  
<br>

1.   Finding $B$ from `equations_function(X)`:  
    If X is a vector of all zeros, then $A \cdot 0 - B = -B$. <br>So, calling equations_function with an $X$ of all zeros will directly return $(- B)$ to you
2.   Finding columns of $A$:
    Consider how `equations_function(X)` behaves when only one element of $X$ is 1 and the rest are 0.  
    For example if $X = [1, 0]$, then `equations_function([1, 0])` returns  
$A \cdot \begin{bmatrix} 1 \\ 0  \end{bmatrix} - B =  \begin{bmatrix}  a_{11} & 0 \\ a_{21} & 0 \end{bmatrix}-B$

<br><br>
This result is the first column of $A$ minus $B$.
So:


*   knowing $B$ already, (`equations_function([1, 0]) + B`) will yield the first column of $A$.
*   Similarly, (`equations_function([0, 1]) + B`) will yield the second column of $A$.  

<br>


Here's "brute force" code for that -- remember your loops:

In [3]:
# --- Step 4: Constructing A and b ---

num_unknowns = 2 # We have two unknowns, x and y

# 4.1: Construct Vector b
X_zeros = np.zeros(num_unknowns)        #Initializing with all zeros
b = -simple_equations(X_zeros)          # determining b
print(f"\nb (from equations_function(X_zeros)): {b}")

# 4.2: Construct Matrix A
A_matrix = np.zeros((num_unknowns, num_unknowns)) # Initialize an empty A matrix

for i in range(num_unknowns):               # looping over # of unknowns
    X_dummy = np.zeros(num_unknowns)
    X_dummy[i] = 1.0 # Set the i-th unknown to 1, others to 0

    # Calculate the corresponding column of A
    A_matrix[:, i] = simple_equations(X_dummy) + b    # fills all of column i

print(f"Constructed Coefficient Matrix A:\n{A_matrix}")


b (from equations_function(X_zeros)): [7. 1.]
Constructed Coefficient Matrix A:
[[ 2.  3.]
 [ 1. -1.]]


## Step 5: Solving with numpy.linalg.solve

NumPy provides a function, `np.linalg.solve(A, b)`, to solve systems of linear equations in the standard form $A \cdot X = B$.  


*   You created $B$ from Step 4 computationally.
*   You have $Amatrix$ from Step 4 computationally.

In [4]:
# --- Step 5: Solving with numpy.linalg.solve ---

X_numpy_solution = np.linalg.solve(A_matrix, b)

print(f"\nSolution using numpy.linalg.solve: {X_numpy_solution}")
print(f"x = {X_numpy_solution[0]:.2f}")
print(f"y = {X_numpy_solution[1]:.2f}")


Solution using numpy.linalg.solve: [2. 1.]
x = 2.00
y = 1.00


## Step 7: Comparing Solutions

Finally, compare the results to ensure consistency.

In [5]:
# --- Step 6: Comparing Solutions ---

print("\n--- Solution Comparison ---")
print(f"Manual Solution (x, y): {X_manual[0]:.2f}, {X_manual[1]:.2f}")
print(f"NumPy Solution (x, y): {X_numpy_solution[0]:.2f}, {X_numpy_solution[1]:.2f}")

# Using np.allclose() is best for comparing floating-point numbers
if np.allclose(X_manual, X_numpy_solution):
    print("The manual and NumPy solutions are in excellent agreement!")
else:
    print("There is a discrepancy between the manual and NumPy solutions.")




--- Solution Comparison ---
Manual Solution (x, y): 2.00, 1.00
NumPy Solution (x, y): 2.00, 1.00
The manual and NumPy solutions are in excellent agreement!


This tutorial covered the use of the `equations_function(X)` and the programmatic construction of $A$ and $B$ as key skills.

## Advantages of the programmatic approach:  
Defining equations in a function and programmatically building $A$ and $B$ matrices,then using `numpy.linalg.solve`, makes solving systems of linear equations much more efficient and less prone to errors for larger or more complex problems than solving by hand.  

You just write all your equations in residual form with little or no algebra, as long as they are linear; this keeps the focus on ChemE not math.

It also allows you to quickly and programmatically test different inputs or other parameters.

# Before we leave!
Why stop there when we can generalize matrix generation in a reuseable function?

Below is a basic general-purpose function that will create matrices for well-defined matrices (we'll extend it next week to well-defined problems).
Interestingly, when you call this function, the name of the function your "rearranged" equations are in, is what you call the function with. And before I sign off, now that you're familiar with this, we'll try and package it so you can import it like a library:

In [6]:
def makematrix(func,N):
  """
  Numerical method for generating a matrix for a linear system of equations
  First b is determined by setting all variables values to 0
  then A is formed column by column by setting one variable at a time to 1

  It takes two input arguments
    1. the function name with the linear equations
    2. the number of unknowns (note number of equations can be different)

  It performs a couple of basic checks
    1. All-zero column, means you have an unneeded unknown or errors
    2. All-zero row, an unneeded equation or errors
    3. Linearity.  By testing a multiple, do you get the same multiple of the result vector
  """

  dummy = np.zeros(N) # initializing a vector with N elements
  b = -func(dummy)              # determine b
  m = len(b)                      # can have different equations != N
  print(f"# of equations m = {m} and # of unknowns N = {N}")
  if m != N:
    if m < N:
      print (f"too few equations for N = {N}")
    else:
      print ("too many equations for N")

  A = np.zeros((m,N))             # create a placeholder A with zeros
  for j in range(N):              # proceed columnwise for every unknown
    dummy = np.zeros(N)           # initializing dummy to all zeros
    dummy[j] = 1                  # setting unknown to 1
    A[:,j] = func(dummy) + b    # computing column j of A

    is_all_zeros = np.all(A[:,j] == 0)  # check - you don't want an all-zero column!
    if is_all_zeros:
      print(f"Bad matrix, column {j +1}  is all zeros.")

  for i in range(m):
    is_all_zeros = np.all(A[i,:] == 0)
    if is_all_zeros:
      print(f"Bad matrix, row {i+1} is all zeros.")    # another bad matrix check

  # a linearity test
  testlin1 = func(np.ones(N)) + b
  testlin2 = (func(10*np.ones(N)) + b)/10
  testlin = np.allclose(testlin1,testlin2,atol = 1e-10)
  if testlin == False:
    print ("function contains nonlinear equations")


  return A,b

## Using `makematrix`

In [20]:
A,b = makematrix(simple_equations,2)  # we gave it 2 as the # of unknowns
print(f"Matrix A:\n{A}")
print(f"Vector b:\n{b}")

X_numpy_solution = np.linalg.solve(A_matrix, b)

print(f"\nSolution using numpy.linalg.solve: {X_numpy_solution}")
print(f"x = {X_numpy_solution[0]:.2f}")
print(f"y = {X_numpy_solution[1]:.2f}")

# of equations m = 2 and # of unknowns N = 2
Matrix A:
[[ 2.  3.]
 [ 1. -1.]]
Vector b:
[7. 1.]

Solution using numpy.linalg.solve: [2. 1.]
x = 2.00
y = 1.00


In [31]:
def forward(X):
    """
    A 4x4 problem cumbersome by hand
    Forward Water Inc forward osmosis unit
    Purpose: extract useful water out of the surprisingly large amount of saltwater produced in oil extraction
    4 streams
      in
        salty feed water (3.5% salt S in water W) - m1
        dried "draw solution (35% hygroscopic stuff H, rest W) - m2
      out
        concentrated brine (send back underground - 10% S, 2.5% H) - m3
        wet "draw solution" (20% H, 0.1% S) - m4

   Question:  water recovery rate?

   Need to solve for m1 through m4
   Balances in the form in = out, since no reaction
   total
      m1  + m2  = m3  + m4
  salt
      0.035m1 = 0.1 m3 + 0.001 m4
  hygroscopic
      0.35m2 = 0.2 m4 + 0.025 m3
  should we add water?

    """
    m1, m2, m3, m4 = X # this is pretty sweet, with 4 unknowns

    m = 4 # 3 independent balance equations total, salt, draw chemical
    eq = np.zeros(m)

    # material balances, open system CV, steady-state, no reactions
    #eq   = (ins)         - (outs)                    # material balance
    eq[0] = (m1  + m2)    - (m3  + m4)                # Total
    eq[1] = (0.035*m1)    - (0.1* m3 + 0.001* m4)     # Salt
    eq[2] = (0.35*m2)     - (0.2* m4 + 0.025* m3)     # Draw Solution
    eq[3] = 0

    return eq



In [32]:
"""
make matrices and solve the problem
"""
A,b = makematrix(forward,4)  # we gave it 4 as the # of unknowns
print(f"Matrix A:\n{A}")
print(f"Vector b:\n{b}")

# A = np.vstack([A, [0,0,0,0]])
# b = np.append(b, 0)
# print(A.shape, b.shape)

solution = np.linalg.solve(A, b)

# Set print options to limit to 2 decimal places
np.set_printoptions(precision=2)
print(f"\nSolution using numpy.linalg.solve: {solution}")

# of equations m = 4 and # of unknowns N = 4
Bad matrix, row 4 is all zeros.
Matrix A:
[[ 1.     1.    -1.    -1.   ]
 [ 0.035  0.    -0.1   -0.001]
 [ 0.     0.35  -0.025 -0.2  ]
 [ 0.     0.     0.     0.   ]]
Vector b:
[-0. -0. -0. -0.]


LinAlgError: Singular matrix

### Very last note:  
you don't need to copy that massive makematrix function, but it's good you know what it does.

We've put it in tenten_v1.py (we will be updating it with more features once we learn about them)

So upload tenten_v1.py into your working files as you did with my_data.txt last week, and then put the following one line of code at the top of your script, similar to importing numpy and so on

Then you can just focus on creating a function with your material and energy balances and constraints almost exactly as you would write them by hand.


Computing solutions literally becomes two lines of code after that.  Of course you need to output the solution as well, though.

In [14]:
from tenten_v1 import makematrix   # make sure to add tenten_v1.py to your active files, and put this at the top of your script
# normally you'd define your mass and energy balances first, i.e. the problems you're trying to solve

"""
then solve the problems.  make matrices and compute the solution
"""
A,b = makematrix(forward,4)  # we gave it 4 as the # of unknown
solution = np.linalg.solve(A, b)

# Set print options to limit to 2 decimal places
np.set_printoptions(precision=2)
print(f"\nSolution using numpy.linalg.solve: {solution}")

# of equations m = 3 and # of unknowns N = 4
too few equations for N = 4


LinAlgError: Last 2 dimensions of the array must be square